In [5]:
!pip install pyarrow fastparquet

  Using cached pyarrow-16.0.0-cp38-cp38-manylinux_2_28_x86_64.whl (40.8 MB)
  Using cached fastparquet-2024.2.0-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.7 MB)
  Using cached cramjam-2.8.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.0 MB)


In [1]:
from IPython.display import Image, Math
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

# Импортируем библиотеки для визуализаци данных
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import logging
logging.basicConfig()
logger = logging.getLogger("model")
logger.setLevel(logging.INFO)

# 3. Оптимизация

In [2]:
import logging
from typing import List, Dict, Any

import numpy as np
import pandas as pd

logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)


def log_uplifts(
    constraints: Dict[str, float],
    maximized_column: str,
    optimal_statistics: Dict[str, float],
) -> None:
    """
    Функция для логирования значений метрик и их аплифтов (улучшений).

    :param constraints: Словарь ограничений для метрик.
    :param maximized_column: Название столбца, который подлежит максимизации.
    :param optimal_statistics: Словарь с оптимальными статистическими данными.
    """
    # Логируем значение метрики, которую мы максимизируем
    logger.info(f"Metric: {maximized_column}", extra={"value": optimal_statistics.get(maximized_column)})

    # Проходим по всем метрикам и их ограничениям
    for metric, constraint in constraints.items():
        optimal_value = optimal_statistics.get(metric)
        if optimal_value is None:
            raise ValueError(f"`{metric}` has not been counted")
        # Логируем информацию по каждой метрике, включая аплифты
        log_dict = {
            "constraint value": round(constraint, 3),
            "optimal value": round(optimal_value, 3),
            "uplift (abs)": round(optimal_value - constraint, 3),
            "uplift (pct)": round(optimal_value * 100 / constraint - 100, 3),
        }
        logger.info(f"Metric: {metric}")
        for key, value in log_dict.items():
            logger.info(f"{key}= {value}")


def apply_constraints(
    df: pd.DataFrame,
    constraints: Dict[str, float],
) -> pd.DataFrame:
    """
    Фильтруем датасет по заданным ограничениям.

    :param df: DataFrame с данными для фильтрации.
    :param constraints: Словарь ограничений для каждой метрики.
    :return: Отфильтрованный DataFrame.
    """
    # Применяем ограничения к датафрейму, фильтруя строки
    for metric, constraint in constraints.items():
        df = df[df[metric] >= constraint]
    return df


def calculate_cum_lambda_metrics(
    df: pd.DataFrame,
    agg_columns: List[str],
    maximized_column: str,
) -> pd.DataFrame:
    """
    Считаем агрегированные значения метрик для каждой комбинации лямбда-значений.

    :param df: DataFrame с данными.
    :param agg_columns: Список столбцов для агрегации.
    :param maximized_column: Столбец, который максимизируется.
    :return: Агрегированный DataFrame.
    """
    # Группируем данные по комбинации лямбда-значений и агрегируем указанные столбцы
    df = df.groupby("lambda_combination").agg({column: "sum" for column in agg_columns})
    df = df.reset_index()
    return df


def choose_optimal_values(
    metric_lambda_map: Dict[str, float],
    df: pd.DataFrame,
    levels: List[str],
    price_column: str,
    maximized_column: str,
) -> pd.DataFrame:
    """
    Находим оптимальные цены / наценки для каждого уровня для lambda_value
    """
    # Считаем лагранжианы при lambda_value
    df["lagrangian"] = df[maximized_column]
    lambda_combination_name = ""
    for metric, metric_lambda in metric_lambda_map.items():
        df["lagrangian"] += df[metric] * metric_lambda
        lambda_combination_name += f"{metric}={metric_lambda}_"
    # Находим максимальный лагранжиан для каждого уровня
    optimal_df = df.groupby(levels).agg({"lagrangian": "max"})
    df = df.merge(optimal_df, on=levels + ["lagrangian"], how="inner")
    # Добавляем колонку с lambda_value для запоминания
    df["lambda_combination"] = lambda_combination_name.strip("_")
    # Удаляем дубликаты (например, оставляем минимальные цены / наценки из оптимальных),
    # так как возможны одни и те же значения метрик для разных цен / наценок
    # => одинаковые лагранжианы, а нам нужно выбрать одно значение для каждого уровня
    df = df.sort_values(price_column)
    df = df.drop_duplicates(subset=levels)
    return df


def get_metric_lambda_maps(lambda_config: Dict[str, Any]) -> List[Dict[str, float]]:
    # Получаем список значений для каждого ключа
    lambda_lists = list(lambda_config.values())
    # Используем meshgrid для генерации всех комбинаций параметров
    lambda_mesh = np.meshgrid(*lambda_lists)
    # Преобразование в массив и решейпинг
    lambda_vars = np.stack(lambda_mesh, axis=-1).reshape(-1, len(lambda_config))
    # Создаем список словарей
    metric_lambda_maps = [
        dict(zip(lambda_config.keys(), combination)) for combination in lambda_vars
    ]
    return metric_lambda_maps


def calculate_lagrangians(
    df: pd.DataFrame,
    lambda_config: Dict[str, Any],
    levels: List[str],
    price_column: str,
    maximized_column: str,
) -> pd.DataFrame:
    """
    Для каждого значения lambda находим оптимальные цены / наценки для каждого уровня
    """
    lambda_dfs = []
    metric_lambda_maps = get_metric_lambda_maps(lambda_config=lambda_config)
    logger.info(
        f"Start calculating lagrangians, {len(metric_lambda_maps)} lambda combinations"
    )
    for metric_lambda_map in metric_lambda_maps:
        lambda_df = choose_optimal_values(
            metric_lambda_map=metric_lambda_map,
            df=df,
            levels=levels,
            price_column=price_column,
            maximized_column=maximized_column,
        )
        lambda_dfs.append(lambda_df)
    df = pd.concat(lambda_dfs)
    df = df.reset_index(drop=True)
    logger.info(f"Ended calculating lagrangians")
    return df


# Общая функция для оптимизации
def optimize(
    df: pd.DataFrame,
    lambda_config: Dict[str, Any],
    maximized_column: str,
    constraints: Dict[str, float],
    levels: List[str],
    price_column: str,
) -> pd.DataFrame:
    logger.info("Start choosing optimal prices")
    lambda_df = calculate_lagrangians(
        df=df,
        lambda_config=lambda_config,
        levels=levels,
        price_column=price_column,
        maximized_column=maximized_column,
    )
    statistics_df = calculate_cum_lambda_metrics(
        df=lambda_df,
        agg_columns=[maximized_column] + list(constraints.keys()),
        maximized_column=maximized_column,
    )
    statistics_df = statistics_df.sort_values(maximized_column, ascending=False)
    logger.info(f"\n{statistics_df.head()}")
    statistics_df = apply_constraints(df=statistics_df, constraints=constraints)
    logger.info(f"\n{statistics_df.head()}")
    best_lambda = statistics_df["lambda_combination"].tolist()[0]
    optimal_statistics = statistics_df[
        statistics_df["lambda_combination"] == best_lambda
    ].to_dict(orient="records")[0]
    optimal_df = lambda_df[lambda_df["lambda_combination"] == best_lambda]
    log_uplifts(
        constraints=constraints,
        maximized_column=maximized_column,
        optimal_statistics=optimal_statistics,
    )
    logger.info("Ended choosing optimal prices")
    return optimal_df

In [8]:
# Считаем ограничения
df = pd.read_csv('./to_karp_5_3.csv')
control_revenue = df[df["discount"] == 0][["margin", "gmv"]].sum()

optimal_df = optimize(
    df=df,
    # перебираем разные lambda для выручки
    lambda_config={
        "margin": np.arange(0.0, 1.1, 0.1).tolist(),
        "gmv": np.arange(0.0, 1.1, 0.1).tolist()
    },
    # указываем, что хотим максимизировать
    maximized_column="orders_num",
    # указываем ограничения
    constraints={
        "margin": control_revenue["margin"],
        "gmv": control_revenue["gmv"]
    },
    levels=["sku_id", "ds"],
    price_column="discount",
)

INFO:__main__:Start choosing optimal prices
INFO:__main__:Start calculating lagrangians, 121 lambda combinations
INFO:__main__:Ended calculating lagrangians
INFO:__main__:
                   lambda_combination    orders_num  margin           gmv
0                  margin=0.0_gmv=0.0  3.357924e+06     0.0  1.976511e+08
6   margin=0.0_gmv=0.6000000000000001  3.357924e+06     0.0  1.976511e+08
10                 margin=0.0_gmv=1.0  3.357924e+06     0.0  1.976511e+08
9                  margin=0.0_gmv=0.9  3.357924e+06     0.0  1.976511e+08
8                  margin=0.0_gmv=0.8  3.357924e+06     0.0  1.976511e+08
INFO:__main__:
                   lambda_combination    orders_num        margin  \
21                 margin=0.1_gmv=1.0  3.309405e+06  3.426072e+07   
19                 margin=0.1_gmv=0.8  3.308614e+06  3.426947e+07   
20                 margin=0.1_gmv=0.9  3.308614e+06  3.426947e+07   
16                 margin=0.1_gmv=0.5  3.308614e+06  3.426947e+07   
17  margin=0.1_gmv=0.600

In [9]:
# Посмотрим распределение скидок
optimal_df["discount"].value_counts(normalize=True)

 0.1    0.924566
-0.1    0.075434
Name: discount, dtype: float64

In [10]:
optimal_df.sort_values(['sku_id', 'ds'])

,sku_id,discount,orders_num,gmv,margin,ds,lagrangian,lambda_combination
166695,1,0.1,2320.362976,31833.838469,6366.767694,20240101,34790.878215,margin=0.1_gmv=1.0
167102,1,0.1,2331.617025,32563.472254,6512.694451,20240102,35546.358724,margin=0.1_gmv=1.0
167670,1,0.1,2422.860191,37409.467858,7481.893572,20240103,40580.517406,margin=0.1_gmv=1.0
167428,1,0.1,2501.122012,41366.607558,8273.321512,20240104,44695.061721,margin=0.1_gmv=1.0
166583,1,0.1,2574.614243,41224.684283,8244.936857,20240105,44623.792212,margin=0.1_gmv=1.0
...,...,...,...,...,...,...,...,...
167429,401,0.1,1025.330837,104133.540277,20826.708055,20240103,107241.541919,margin=0.1_gmv=1.0
166584,401,0.1,1047.055648,124292.715808,24858.543162,20240104,127825.625772,margin=0.1_gmv=1.0
166526,401,0.1,1037.700417,114523.808942,22904.761788,20240105,117851.985538,margin=0.1_gmv=1.0
167005,401,0.1,1077.292167,118172.713031,23634.542606,20240106,121613.459458,margin=0.1_gmv=1.0


In [11]:
optimal_df[[
    'sku_id',
    'discount',
    'ds'
]].to_csv('to_karp_5_4.csv', index=False)

In [12]:
# Считаем аплифт на предсказаниях
control_margin = df[df["discount"] == 0]["margin"].sum()
control_revenue = df[df["discount"] == 0]["gmv"].sum()

test_margin = optimal_df["margin"].sum()
test_revenue = optimal_df["gmv"].sum()

print(f"Control margin: {round(control_margin)} руб.")
print(f"Test margin: {round(test_margin)} руб.")
print(f"Uplift: {round(test_margin * 100 / control_margin - 100)} %")

print(f"Control revenue: {round(control_revenue)} руб.")
print(f"Test revenue: {round(test_revenue)} руб.")
print(f"Uplift: {round(test_revenue * 100 / control_revenue - 100)} %")

Control margin: 19368542 руб.
Test margin: 34260721 руб.
Uplift: 77 %
Control revenue: 193685421 руб.
Test revenue: 197303312 руб.
Uplift: 2 %
